In [ ]:
from pypge.search import PGE
from pypge import expand
from pypge.benchmarks import explicit

import numpy as np

# visualization libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# plot the visuals in ipython
%matplotlib inline


In [ ]:
prob = explicit.Nguyen_12(0.1)
print prob['name'], prob['eqn']
print prob['xpts'].shape

plt.plot(prob['xpts'][0], prob['ypure'], 'r.')
plt.show()
plt.plot(prob['xpts'][0], prob['ypts'], 'b.')
plt.show()

In [ ]:
pge = PGE(
    system_type = "explicit",
    search_vars = "y",
    usable_vars = prob['xs_str'],
    usable_funcs = expand.BASIC_BASE[1:],
    pop_count = 3,
    max_iter = 10
    )

pge.fit(prob['xpts'], prob['ypts'])


In [ ]:
print len(pge.final)
print "\n"
for m in pge.final_paretos[0]:
    print m

In [ ]:
pge_szs = [m.size() for m in pge.final]
pge_scr = [m.score for m in pge.final]
pge_evar = [1.0 - m.evar for m in pge.final]

pge_szs_f = [m.size() for m in pge.final_paretos[0]]
pge_scr_f = [m.score for m in pge.final_paretos[0]]
pge_evar_f = [1.0 - m.evar for m in pge.final_paretos[0]]

plt.plot(pge_szs, pge_scr, 'b.', pge_szs_f, pge_scr_f, 'ro')
plt.show()
plt.plot(pge_szs, pge_evar, 'b.', pge_szs_f, pge_evar_f, 'ro')
plt.show()

In [ ]:
from pypge.evaluate import Eval

for best_m in pge.final_paretos[0]:
    print best_m
    y_pred = Eval(best_m, pge.vars, prob['xpts'])
    plt.plot(prob['xpts'][0], prob['ypts'], 'r.',prob['xpts'][0], y_pred, 'b.')
    plt.show()


In [ ]:
from sympy import *

import networkx as nx
G = pge.graph
n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()
print n_nodes, n_edges

print nx.info(G)

print nx.density(G)

bins = nx.degree_histogram(G)


# pos=nx.graphviz_layout(G,prog="twopi",root=pge.root_model)
# nx.draw_networkx(G,pos,with_labels=False,node_size=30)
# nx.draw_circular(G,with_labels=False,node_size=30)

# pos=nx.shell_layout(G,pge.iter_expands)
# nx.draw_networkx(G,pos,with_labels=False,node_size=30)

plt.yscale('log')
plt.bar(range(0,len(bins)),bins)
plt.draw()
